In [7]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh


In [3]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE


# setup environment GEITje-7B Finetuning
# - pip install torch
# - pip install datasets
# - pip install transformers
# - pip install trl
# - pip install accelerate (restart after)
# - switch device_map='auto' to avaoid memory error

# - pip install sentencepiece
# - pip install jupyter
# - pip install protobuf 



## Notebook overview
This notebook creates predictions for the baseline models. In total, five models are tried out.
- Training function. Given a baseline model, will return scores.
- Load Data. Load all the documents, and set parameters.
- save predictions


Kernel: Pytorch and Tensorflow

### Load file with training funcation


In [4]:
import sys
sys.path.append('../scripts/') 
import baseline as bf

### Load data

In [9]:
from collections import Counter
import pandas as pd

# df = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")
df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")

print(Counter(df['2split']))
print(Counter(df['4split']))

#set  variables, same for each model
TRAIN_SET = 'train' # must be dev or train
TEST_SET = 'test' # must be val or test
SPLIT_COLUMN = '4split' #must be either 2split or 4split. 2split = data split into train and test. 4split = data split into train, test, dev and val. 
TEXT_COLUMN = 'text'
LABEL_COLUMN = 'label'
DATAFRAME = df.copy()
PATH = f"{cf.output_path}/predictions/baselinePredictions.pkl"
OVERVIEW_PATH = f"{cf.output_path}/overview/baselineOverview.pkl"

### Baselines

##### Baseline 1: linear SVM+tf-idf

In [ ]:
from sklearn.svm import LinearSVC
linear_svm = bf.run_baseline(LinearSVC(), 'LinearSVC' , DATAFRAME, SPLIT_COLUMN, TRAIN_SET, TEST_SET,TEXT_COLUMN, LABEL_COLUMN, PATH, OVERVIEW_PATH)

                      precision    recall  f1-score   support

         Actualiteit       0.98      0.84      0.90       152
              Agenda       0.98      0.98      0.98       528
             Besluit       0.96      0.94      0.95       113
               Brief       0.91      0.88      0.90       206
          Factsheets       0.91      0.47      0.62        45
               Motie       0.98      0.98      0.98      1545
   Onderzoeksrapport       0.80      0.97      0.88       222
          Raadsadres       0.91      0.98      0.94       313
        Raadsnotulen       1.00      1.00      1.00        42
Schriftelijke Vragen       0.99      0.95      0.97       603
          Voordracht       1.00      1.00      1.00       395

            accuracy                           0.96      4164
           macro avg       0.95      0.91      0.92      4164
        weighted avg       0.96      0.96      0.96      4164



##### Baseline 2: Naive Bayes+tf-idf

In [ ]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = bf.run_baseline(MultinomialNB(), 'MultinomialNB' , DATAFRAME, SPLIT_COLUMN, TRAIN_SET, TEST_SET,TEXT_COLUMN, LABEL_COLUMN, PATH, OVERVIEW_PATH)

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

                      precision    recall  f1-score   support

         Actualiteit       0.00      0.00      0.00       152
              Agenda       0.98      0.92      0.95       528
             Besluit       0.00      0.00      0.00       113
               Brief       0.00      0.00      0.00       206
          Factsheets       0.00      0.00      0.00        45
               Motie       0.46      1.00      0.63      1545
   Onderzoeksrapport       0.38      0.06      0.11       222
          Raadsadres       1.00      0.00      0.01       313
        Raadsnotulen       0.00      0.00      0.00        42
Schriftelijke Vragen       1.00      0.05      0.09       603
          Voordracht       0.99      0.66      0.79       395

            accuracy                           0.56      4164
           macro avg       0.44      0.24      0.23      4164
        weighted avg       0.63      0.56      0.45      4164



/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision, recall and f1-score equal to zero occurs if there are not True Positives. Meaning for those classes not one document is correctly predicted. 

##### Baseline 3: Logistic Regression + tf-idf

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = bf.run_baseline(LogisticRegression(), 'LogisticRegression' , DATAFRAME, SPLIT_COLUMN, TRAIN_SET, TEST_SET,TEXT_COLUMN, LABEL_COLUMN, PATH, OVERVIEW_PATH)

KeyboardInterrupt: 

##### Baseline 4: k Nearest Neigbors + tf-idf

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = bf.run_baseline(KNeighborsClassifier(), 'KNeighborsClassifier' , DATAFRAME, SPLIT_COLUMN, TRAIN_SET, TEST_SET,TEXT_COLUMN, LABEL_COLUMN, PATH, OVERVIEW_PATH)

                      precision    recall  f1-score   support

         Actualiteit       0.62      0.39      0.47       205
              Agenda       0.74      0.95      0.83       696
             Besluit       0.74      0.86      0.80       162
               Brief       0.40      0.58      0.47       408
          Factsheets       0.64      0.39      0.48        41
               Motie       0.86      0.66      0.74      1713
   Onderzoeksrapport       0.25      0.67      0.36       258
          Raadsadres       0.85      0.26      0.40       408
        Raadsnotulen       0.17      0.98      0.30        58
Schriftelijke Vragen       0.72      0.44      0.55       557
       Termijnagenda       0.74      0.44      0.55       189
          Voordracht       0.89      0.84      0.87       645

            accuracy                           0.65      5340
           macro avg       0.64      0.62      0.57      5340
        weighted avg       0.74      0.65      0.66      5340



##### Baseline 5: RandomForest + tf-idf

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest = bf.run_baseline(RandomForestClassifier(), 'RandomForestClassifier' , DATAFRAME, SPLIT_COLUMN, TRAIN_SET, TEST_SET,TEXT_COLUMN, LABEL_COLUMN, PATH, OVERVIEW_PATH)

                      precision    recall  f1-score   support

         Actualiteit       0.92      0.71      0.80       205
              Agenda       0.85      0.89      0.87       696
             Besluit       0.98      0.89      0.93       162
               Brief       0.90      0.90      0.90       408
          Factsheets       0.48      0.32      0.38        41
               Motie       0.96      0.97      0.96      1713
   Onderzoeksrapport       0.84      0.90      0.87       258
          Raadsadres       0.85      0.95      0.90       408
        Raadsnotulen       1.00      0.95      0.97        58
Schriftelijke Vragen       0.98      0.94      0.96       557
       Termijnagenda       0.55      0.46      0.50       189
          Voordracht       0.98      0.99      0.99       645

            accuracy                           0.91      5340
           macro avg       0.86      0.82      0.84      5340
        weighted avg       0.91      0.91      0.91      5340



### Overview of all runs

In [ ]:
overview = pd.read_pickle(OVERVIEW_PATH)
display(overview)